In [22]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import uuid
import os
import urllib
import json
from json import JSONEncoder
from uuid import UUID

global Postcode
Postcode = "BT5"

class Scraper:
    def __init__(self, url: str = 'https://www.propertypal.com') -> None:
        self.driver = Chrome(ChromeDriverManager().install())
        self.driver.get(url)
        
    def ivan_accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        
        try:
            time.sleep(1)
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))

            button = self.driver.find_element(By.XPATH, xpath)
            button.click()
        except TimeoutException:
            print("No Cookies Found")

    def accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        button = self.driver.find_element(By.XPATH, xpath)
        button.click()

    
    def search_word(self, xpath, pc = Postcode):

        search = self.driver.find_element(By.XPATH, xpath)
        self.driver.implicitly_wait(10)
        ActionChains(self.driver).move_to_element(search).click(search).perform()
        ActionChains(self.driver).send_keys(pc).perform()

    def scroll_down(self):
        
        self.driver.execute_script("window.scrollTo(0, 540)") 
    
    def search_rent(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[2]')
        rent.click()

    def search_sale(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[1]')
        rent.click()

    def button_click(self,xpath):
        button = self.driver.find_element_by_xpath(xpath)
        button.click()
    
    def find_container(self, xpath: str = '//*[@id="body"]/div[3]/div/div[1]/div/ul'):
        self.container = self.driver.find_element(By.XPATH, xpath)
        return self.container

##
# NAVIGATE THE WEBSITE
##

if __name__ == "__main__":
    bot = Scraper()
    bot.ivan_accept_cookies()
    time.sleep(2)
    bot.button_click('//a[@href="/login"]')
    time.sleep(2)
    bot.search_word('//input[@placeholder="Email address"]','sopranotony233@gmail.com')
    time.sleep(2)
    bot.search_word('//input[@placeholder="Password"]','sopranotony321')
    time.sleep(2)
    bot.button_click('//*[@id="root"]/div/div/div/div/div/div[1]/div/div[2]/div[2]/form/button')
    time.sleep(2)
    bot.button_click('//a[@class="mainnav-logo"]')  
    time.sleep(2)
    bot.search_word('//*[@id="searchForm"]/div/div[1]')
    time.sleep(2)
    bot.search_rent()
    time.sleep(2)
    

    #CREATE THE LIST OF LINKS
    list_links = []
    print("Finding elements...")
    while True:
        container = bot.find_container()
        items = container.find_elements(By.XPATH, './li')
        for i in items:
            try:
                house = i.find_element(By.XPATH, './/a[2]')
                link = house.get_attribute('href')
                list_links.append(link)
            except:
                print("no href found")
        
        try:
            bot.button_click('//a[@class="btn paging-next"]')
        except NoSuchElementException:
            print("end of list")
            break

    #GRAB INFO FROM EACH LINK AND STORE
    print("Grabbing info...")
    prop_dict = {"fr-id": [],
            "id": [],
            "Link": [],
             "Summary": [],
            "Address": [],
            "Price": [],
            "Image links": []
            }
    for link in list_links:
        im = link[-6:]
        im2 = link[28:35]
        prop_dict["fr-id"].append(im2+im)
        id = uuid.uuid4()
        prop_dict["id"].append(id)
        bot.driver.get(link)
        prop_dict["Link"].append(link)
        time.sleep(0.5)
        summary = bot.driver.find_element(By.XPATH, '//div[@class="prop-heading-brief"]')
        prop_dict["Summary"].append(summary.text)
        time.sleep(0.5)
        info = bot.driver.find_element(By.XPATH, '//div[@class="prop-summary-row"]')
        address = info.find_element(By.XPATH, './/h1')
        prop_dict["Address"].append(address.text)
        time.sleep(0.5)
        price = bot.driver.find_element(By.XPATH, '//div[@class="prop-price"]')
        prop_dict["Price"].append(price.text)
        img_list = bot.driver.find_elements(By.XPATH, '//div[@class="Slideshow-slides SlideshowCarousel"]//img')
        img_links = []

        
        for img in img_list:
            try:
                link = img.get_attribute('src')
                img_links.append(link)
            except:
                print('No src found')
        prop_dict["Image links"].append(img_links)


    os.mkdir(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos/{Postcode}")
    image_directory = os.path.dirname(f"/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/Property_Photos/{Postcode}/")
    img_link_ct= -1
    for img_list in prop_dict["Image links"]:
        img_link_ct += 1
        img_ct = 0
        for url in img_list:
            try:
                img_ct += 1
                req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                with open(f"{image_directory}/{prop_dict['fr-id'][img_link_ct]}_{str(img_ct)}.jpg", "wb") as f:
                    with urllib.request.urlopen(req) as r:
                        f.write(r.read())
            except IndexError:
                pass 

    old_default = JSONEncoder.default

    def new_default(self, obj):
        if isinstance(obj, UUID):
            return str(obj)
        return old_default(self, obj)

    JSONEncoder.default = new_default
 
    with open(f'/Users/ryanhughes/Desktop/Aicore/Property-Pal-Pipeline-/raw_data/{Postcode}.json', 'w') as f:
        json.dump(prop_dict, f, indent = 4)

    df = pd.DataFrame(prop_dict)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.134/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_59493/1245743024.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_59493/1245743024.py:64: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_59493/1245743024.py:56: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  rent = self.driver.find_element_by_xpath('//*[@id="

Finding elements...
no href found
no href found
no href found
end of list
Grabbing info...


In [23]:
df

,fr-id,id,Link,Summary,Address,Price,Image links
0,5-elmgr770105,ea41700b-fa89-483e-a48c-1550bc161b4c,https://www.propertypal.com/5-elmgrove-mews-23...,1 Bed 1st Floor Apartment To Rent,"5 ELMGROVE MEWS,\n231-239 beersbridge road, be...",£500\n/ month + fees,[https://media.propertypal.com/sd/p/768905/326...
1,5-brook769768,e504ab72-50ad-41e4-9a12-a80b6d3a91e3,https://www.propertypal.com/5-brookeborough-ha...,2 Bed Apartment To Rent,"5 BROOKEBOROUGH HALL 117 SANDOWN ROAD,\nBelfas...",£775\n/ month + fees,[https://media.propertypal.com/handheld/p/7685...
2,apt-30-769743,8593667d-3c92-429d-a7dd-b001c02b311b,https://www.propertypal.com/apt-30-grafton-cou...,2 Bed Apartment To Rent,"APT 30 GRAFTON COURT, 85 KINGS ROAD,\nBelfast ...",£950\n/ month + fees,[https://media.propertypal.com/sd/p/768543/326...
3,8-elmco769573,7ce22177-3044-411f-9071-1796815bc71f,https://www.propertypal.com/8-elmcourt-315-bee...,1 Bed 1st Floor Apartment To Rent,"8 ELMCOURT,\n315 Beersbridge Road, Belfast BT5...",£525\n/ month + fees,[https://media.propertypal.com/handheld/p/7683...
4,40-kinr769530,0b4807da-8ee4-42e4-bfc5-31acfb301f44,https://www.propertypal.com/40-kinross-avenue-...,3 Bed End-terrace House To Rent,"40 KINROSS AVENUE,\nBelfast BT5 7GH",£650\n/ month + fees,[https://media.propertypal.com/sd/p/768330/325...
5,9-kensi769517,a791232f-5747-4882-8fc0-77d6df8e7bc0,https://www.propertypal.com/9-kensington-avenu...,2 Bed Mid-terrace House To Rent,"9 KENSINGTON AVENUE,\nBelfast BT5 5JN",£800\n/ month + fees,[https://media.propertypal.com/sd/p/768317/325...
6,30-avon769514,cb9be780-1120-4eaf-8dd6-205d3d0362b5,https://www.propertypal.com/30-avondale-street...,2 Bed Mid-terrace House To Rent,"30 AVONDALE STREET,\nBelfast BT5 5DB",£600\n/ month + fees,[https://media.propertypal.com/sd/p/768314/325...
7,apartme548250,65076af7-86b5-4371-a2e6-e89311b32c67,https://www.propertypal.com/apartment-12-202-2...,1 Bed Apartment To Rent,"APARTMENT 12 202-206 ALBERTBRIDGE ROAD,\nBelfa...",£575\n/ month + fees,[https://media.propertypal.com/handheld/p/5470...
8,apt-2-2769411,b2fc16dd-7cf3-4e60-9d45-2ea21964e30c,https://www.propertypal.com/apt-2-2a-bloomdale...,2 Bed Apartment To Rent,"APT 2, 2A BLOOMDALE STREET,\nBloomfield Avenue...","£1,000\n/ month + fees",[https://media.propertypal.com/sd/p/768211/325...
9,apt-1-2769410,53c594f2-b7a2-4036-9ded-6c417c9d6bf4,https://www.propertypal.com/apt-1-2a-bloomdale...,2 Bed Apartment To Rent,"APT 1, 2A BLOOMDALE STREET,\nBloomfield Avenue...","£1,000\n/ month + fees",[https://media.propertypal.com/sd/p/768210/325...


In [4]:
bt4 = [
        "15-holy769842",
        "14-sand769820",
        "80-belm769451",
        "a9-alto769347",
        "9-campb515070",
        "85-uppe769189",
        "3-castl769168",
        "apt-14-768940",
        "27-garn768910",
        "57-vict768648",
        "apt2-4-768598",
        "28-park768346",
        "apartme768044",
        "12-inve768013",
        "68-belm356006",
        "19-asto767775",
        "127-cas767763",
        "apt-9-3767651",
        "9-pims-765427",
        "apt-3-g765358",
        "53-uppe765329",
        "40-norw765201",
        "apartme764434",
        "79-uppe730603",
        "18-stra762633",
        "85-uppe606480",
        "apt-4-4676830",
        "apt-a-5746507"]

In [5]:
len(bt4)

28

In [6]:
bot = Scraper()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.134/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_59493/1180928362.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())


In [7]:
bot.ivan_accept_cookies()

In [8]:
import unittest

class TestCase(unittest.TestCase):
    def test_transform_name(self):
        small_black_shoes = Product('shoes', 'S', 'black')
        expected_value = 'SHOES'
        actual_value = small_black_shoes.transform_name_for_sku()
        self.assertEqual(expected_value, actual_value)

unittest.main(argv=[''], verbosity=0, exit=False)

SyntaxError: invalid syntax (3926837683.py, line 1)

In [9]:
bot = Scraper()


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.134/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.134]
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_59493/1180928362.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())


In [14]:
expected_prop_ct = int(bot.driver.find_element(By.XPATH, '//strong[@class="f-red"]').text)

In [15]:
expected_prop_ct

31

In [16]:
import Property_Web_Scraper

In [18]:
import unittest

In [24]:
class TestPropertyCount(unittest.TestCase):
    def runTest(self):
        actual_property_count = len(prop_dict["fr-id"])
        self.assertEqual(actual_property_count, expected_prop_ct, "incorrect")

In [25]:
unittest.main()

usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument '/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/tmp-57865v7w3C4uKlhut.json'


AssertionError: 